In [ ]:
import os
import numpy as np
import time
import pickle
import matplotlib.pyplot as plt
import matplotlib
from sklearn.metrics import accuracy_score, f1_score, classification_report
from scipy.stats import norm
from itertools import cycle
from random import shuffle
from sklearn.model_selection import train_test_split


import torch
from torch import nn
import torch.optim as opt
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import MNIST
from torchvision import transforms
from livelossplot import PlotLosses
from mpl_toolkits.axes_grid1 import ImageGrid
from collections import OrderedDict

from cycle_consistent_vae import Encoder, Decoder

np.random.bit_generator = np.random._bit_generator

In [ ]:
cuda = 1
device = torch.device("cuda:{}".format(cuda) if torch.cuda.is_available() else "cpu")

MODEL_PATH = "./models/cycle_vae_06052020-030456_99.pth"
checkpoint = torch.load(MODEL_PATH)

Z_DIM = 16 #Style Dimension (Unspecified)
S_DIM = 16 # Class Dimension (Specified)

encoder = Encoder(style_dim=Z_DIM, class_dim=S_DIM)
encoder.load_state_dict(checkpoint['encoder'])

decoder = Decoder(style_dim=Z_DIM, class_dim=S_DIM)
decoder.load_state_dict(checkpoint['decoder'])


encoder.to(device)
encoder.eval()
decoder.to(device)
decoder.eval()

In [ ]:
def reparameterize(training, mu, logvar):
    if training:
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return eps * std + mu
    else:
        return mu

# Dataset processing

In [ ]:
class Latent(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.encoder = encoder

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        img = self.data[index][:,:,:3]
        
        return (img, self.labels[index])

In [ ]:
# PATH = './processed/'
# files = os.listdir(PATH)
# data = []
# labels = []

# counter=0
# for folder in files:
#     print(counter)
#     sprites = os.listdir(PATH+folder)
#     for sprite in sprites:
#         data.append(plt.imread(PATH+folder+"/"+sprite))
#         labels.append(int(folder))
#     counter+=1

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(data, labels, \
#                                                     test_size=0.33, random_state=42, \
#                                                     shuffle=True, stratify=labels )

In [ ]:
# train_data = Latent(X_train, y_train)
# test_data = Latent(X_test, y_test)

In [ ]:
# with open('train_latent.pkl','wb') as f:
#     pickle.dump(train_data, f)
# with open('test_latent.pkl','wb') as f:
#     pickle.dump(test_data, f)

In [ ]:
t1 = pickle.load(open('train_latent.pkl','rb'))
t2 = pickle.load(open('test_latent.pkl','rb'))

In [ ]:
BATCH_SIZE = 16
train_loader = DataLoader(t1,batch_size=BATCH_SIZE,shuffle=True,num_workers=2)
test_loader = DataLoader(t2,batch_size=BATCH_SIZE,shuffle=True,num_workers=2)

In [ ]:
NUM_CLASSES=672

# Style tranfer grid